# Mini project: Simulating Pac-Man

In the last practical session, we saw how run multiple behaviors in parallel on multiple robots. In this section we will see more advanced methods for combining behaviors by modulating behavior activations according to internal states of the robot and by allowing them to sense the attributes of others. In order to start with a clean basis, let's first provide the definition of several behaviors. These four behaviors are simple implementations of the [Braitenberg vehicles](https://cdn.rawgit.com/clement-moulin-frier/rti_course/master/class_1/intro.sozi.html#frame1550) we have seen in class, where the sources (i.e. what is sensed by the proximeters) are other epucks:

In [1]:
from numpy.linalg import norm
from numpy import array

from simulator_interface import open_session, close_session
simulator, epuck1, epuck2, epuck3 = open_session(n_epucks=3)

# First reduce the maximum speed
for e in simulator.robots:
    e.max_speed = 3
    e.num_eats = 0
    e.dead = False

def line_following(robot):
    floor_left, floor_middle, floor_right = robot.floor_sensor()
    # if can go right
    if floor_middle == 1.0 and floor_right == 1.0 and robot.rotate == 'right':
        return  [1.0, 0]
    # if can go left
    if floor_middle == 1.0 and floor_left == 1.0 and robot.rotate == 'left':
        return  [0, 1.0]
    if floor_middle == 1.0: return [.3, .3]
    if floor_left == 0.0 and floor_right == 0.0: return [0.1, 0]
    return [floor_right/2.0, floor_left/2.0]

def detect_food(robot):
    left, right = robot.prox_activations(tracked_objects=['Cuboid', 'ePuck#'])
    if (left > 0 or right > 0):
        robot.rotate = 'left' if left > right else 'right'
        
# Reverse-engineered part of the code in order to make spheres disappear when eaten   
def eat(robot): 
    try:
        robot_pos = array(robot.position())
    except:
        print ('robot unavailable')
        robot.stop_all_behaviors()
        robot.detach_all_behaviors()
        robot.stop_all_routines()
        robot.dead = True
        print ('score: ' + str(robot.num_eats))
        return
    for obj in simulator.eatable_objects:
       
        try:
            obj_pos = array(simulator.get_object_position(obj))
            if norm(robot_pos - obj_pos) < 0.05 :
                continue
        except:
            break
        if norm(robot_pos - obj_pos) < 0.1 :
            simulator.eatable_objects.remove(obj)
            if obj.startswith('Cuboid'):
                robot.num_eats = robot.num_eats+1
            else:
                if robot.num_eats==0:
                    continue
            simulator.remove_object(obj)

simulator.eatable_objects = ['Cuboid4','Cuboid0', 'Cuboid1', 'Cuboid2', 'Cuboid3', 'ePuck', 'ePuck#0', 'ePuck#1']
for e in simulator.robots:
    e.detach_all_behaviors()
    e.attach_routine(eat, freq=1)
    e.start_routine(eat)
########

for e in simulator.robots:
    e.detach_all_behaviors()
    e.rotate = 'no'
    e.attach_behavior(line_following, freq=10)
    e.attach_routine(detect_food, freq=1)
    e.start_routine(detect_food)
    e.start_all_behaviors()


    

Routine eat started
Routine eat started
Routine eat started
Routine detect_food started
Behavior line_following started
Routine detect_food started
Behavior line_following started
Routine detect_food started
Behavior line_following started
robot unavailable
Behavior line_following stopped
Routine eat stopped
Routine detect_food stopped
score: 0
robot unavailable
Behavior line_following stopped
Routine eat stopped
Routine detect_food stopped
score: 1


In [2]:
close_session(simulator)

In [ ]:
close_all_connections()